In [1]:
import numpy as np

# function for defining first 5 Seidel polynomials

def seidel_5(u0, v0, X, Y, wd, w040, w131, w222, w220, w311):
    """Compute wavefront OPD for first 5 Seidel wavefront
    aberration coefficients + defocus
    
    u0, v0 - normalized image plane coordinate
    X, Y - normalized pupil coordinate arrays (like from meshgrid)
    wd - defocus, w040 - spherical, w131 - coma, w222 - astigmatism, w220 - field curvature, w311 - distortion
    
    
    """
    beta = np.arctan2(v0, u0)         # image rotation angle
    u0r = np.sqrt(u0**2 + v0**2)      # image height
    
    # rotate grid
    
    Xr = X * np.cos(beta) + Y * np.sin(beta)
    Yr = - X * np.sin(beta) + Y * np.cos(beta)
    
    # Seidel polynomials
    
    rho2 = Xr**2 + Yr**2
    w = (wd * rho2 + 
         w040 * rho2**2 + 
         w131 * u0r * rho2*Xr + 
         w222 * u0r**2 * Xr**2 +
         w220 * u0r**2 * rho2 + 
         w311 * u0r**3 * Xr)   
    return w

def circ(r):
    return (np.abs(r) <= 1).astype(int)

def rect(r):
    return (np.abs(r) <= 0.5).astype(int)

def focus(uin,L,wvl,zf):
    """
     converging or diverging phase-front
     uniform sampling assumed
     uin - input field
     L - side length
     lambda - wavelength
     zf - focal distance (+ converge, - diverge)
    """   
    [M,N] = np.shape(uin)       #  get input field array size
    dx = L/M                    # sample interval
    k = 2*np.pi/wvl             # wavenumber

    x = np.linspace(-L/2, L/2, M)     # coords
    [X,Y] = np.meshgrid(x,x)
    return uin * np.exp(-1j*k/(2*zf)*(X**2 + Y**2))  # apply focus

def get_fresnel_num(w, wvl, z):
    """
    compute Fresnel number for estimation of applicability Fresnel criterion
    w - half-width of a square aperture in the source plane, or the radius of a circular aperture
    wvl - operating wavelength
    z = the distance to the observation plane
    if Nf < 1 for a given scenario - the observation plane in the Fresnel region
    it should describe paraxial assumption - rays with < 0.1 rad angle
    """
    Nf = w**2/(wvl * z)
    return Nf

# Fresnel Transfer Function (TF) Propagator
def propTF(u1,L,wvl,z):
    """
    This propagator function takes the source field u1 and produces the observation
    field u2 where the source and observation side lengths and sample coordinates
    are identical.
    
    propagation - transfer function approach
    assumes same x and y side lengths and
    uniform sampling
    u1 - source plane field
    L - source and observation plane side length
    wvl - wavelength
    z - propagation distance
    u2 - observation plane field
    """
    [M,N] = np.shape(u1)                        # get input field array size
    dx = L/M                                    # sample interval
    k=2*np.pi/wvl                               # wavenumber

    fx = np.linspace(-1/(2*dx), 1/(2*dx), M)    # freq coords
    [FX,FY] = np.meshgrid(fx,fx)
    H = np.exp(-1j*np.pi*wvl*z*(FX**2+FY**2))   # trans func
    H = np.fft.fftshift(H)                      # shift trans func
    U1 = np.fft.fft2(np.fft.fftshift(u1))       # shift, fft src field
    U2=H * U1                                   # multiply
    return np.fft.ifftshift(np.fft.ifft2(U2))   # inv fft, center obs field

# Fresnel Impulse Response (IR) Propagator
def propIR(u1,L,wvl,z):
    """
    Due to computational artifacts, the IR approach turns out to be more limited in
    terms of the situations where it should be used than the TF approach, however, it
    provides a way to simulate propagation over longer distances and is useful for the
    discussion of simulation limitations and artifacts
    
    propagation - impulse response approach
    assumes same x and y side lengths and
    uniform sampling
    u1 - source plane field
    L - source and observation plane side length
    wvl - wavelength
    z - propagation distance
    u2 - observation plane field
    """
    [M,N] = np.shape(u1)                        # get input field array size
    dx = L/M                                    # sample interval
    k=2*np.pi/wvl                               # wavenumber

    x = np.linspace(-L/2, L/2, M)               # spatial coords
    [X,Y] = np.meshgrid(x,x)
    
    h = (1/(1j * wvl * z)) * np.exp(1j * k / (2 * z) * (X**2 + Y**2))   # impulse
    H = np.fft.fft2(np.fft.fftshift(h)) * dx **2                        # create trans func
    U1 = np.fft.fft2(np.fft.fftshift(u1))                               # shift, fft src field
    U2 = H * U1                                                         # multiply
    return np.fft.ifftshift(np.fft.ifft2(U2))                           # inv fft, center obs field

def get_2d_rotated(input_array, output_size):
    if len(2 * input_array) > output_size:
        print('input array length is greater than output image size!')
        return None
    shift = int(0.5 * output_size)
    out = np.zeros((output_size, output_size))
    for i in range(-len(input_array) - 1, len(input_array)):
        for j in range(-len(input_array) - 1, len(input_array)):
            ind = int(np.sqrt(i**2 + j**2))
            if ind < len(input_array):
                out[i + shift][j + shift] = input_array[ind]
    return out
            
def get_normalized_psf(param_dict, aberrations):
    """
    The function calculates normalized PSF by using input parameters describing optical system and aberrations
    
    input parameters - 
    
    param_dict - dictionary with optical system parameters, where
    wvl - wavlenetgh,
    D_lens - lens aperture,
    focal_length - focal length,
    du - sample size in image plane
    size - sensor size
    
    aberrations - can be expressed differently - either as a dictionary with Seidel polynomial values or
    a 2D array representing optical path difference (OPD)
    
    returned parameers - 
    
    Wa - centered wavefront
    Ha - coherent transfer function
    PSF - Point Spread Function
    u - coordinates in the image plane
    fu_i - coordinates (in reciprocal units - m-1) in aperture plane
    """
    
    dictget = lambda d, *k: [d[i] for i in k]
    
    wvl, D_lens, focal_length, du, size = dictget(param_dict, 'wvl', 'D_lens', 'focal_length','du','size')

    k = 2*np.pi / wvl
    num_pix = int(size / du)
    fnum = focal_length / D_lens
    u = np.linspace(-0.5 * size, 0.5 * size, num_pix)        # coordinates in image plane in m units
    v = u
    fN = 1 / (2 * du)                                                # Nyquist frequency
    fu_i = np.linspace(fN, -fN, num_pix)                  # image freq coords
    fu = np.fft.fftshift(fu_i)                            # shift cords, avoid shift H in loop
    [Fu, Fv] = np.meshgrid(fu, fu)

    if (type(aberrations) is dict):
        wd, w040, w131, w222, w220, w311 = dictget(aberrations, 'wd', 'w040', 'w131','w222','w220','w311')
        Wa = seidel_5(u, v, -2 * wvl * fnum * Fu , -2 * wvl * fnum * Fv,
                      wd,w040,w131,w222,w220,w311)  # wavefront
    else:
        # aberrations*=(wvl/(2*np.pi))
        Wa = np.fft.fftshift(aberrations)
        
    Ha = circ(np.sqrt(Fu**2 + Fv**2) * 2 * wvl * fnum) * np.exp(-1j * k * Wa)
    PSF = (np.abs(np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(Ha)))))**2
    PSF = PSF/np.max(PSF)
    
    return np.fft.fftshift(Wa), np.fft.fftshift(Ha), PSF, u, fu_i

    
    
    
